<a href="https://colab.research.google.com/github/shuiyuedongtian123/cma-es-lin-/blob/main/cma_es_with_new_condition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from collections import deque

import numpy as np
import numpy
from deap import algorithms
from deap import base
from deap import benchmarks
from deap import cma
from deap import creator
from deap import tools
import random

MIN_BOUND = [-1, -1, -1, -1, -1]
MAX_BOUND = [1, 1, 1, 1, 1]
EPS_BOUND = 2.e-5

def distance(feasible_ind, original_ind):
    """A distance function to the feasibility region."""
    return sum((f - o)**2 for f, o in zip(feasible_ind, original_ind))

def closest_feasible(individual):
    """A function returning a valid individual from an invalid one."""
    feasible_ind = numpy.array(individual)
    feasible_ind = numpy.maximum(MIN_BOUND, feasible_ind)
    feasible_ind = numpy.minimum(MAX_BOUND, feasible_ind)
    return feasible_ind

def valid(individual):
    """Determines if the individual is valid or not."""
    if any(individual < np.array(MIN_BOUND)) or any(individual > np.array(MAX_BOUND)):
        return False
    return True

def close_valid(individual):
    """Determines if the individual is close to valid."""
    if any(individual < np.array(MIN_BOUND)-EPS_BOUND) or any(individual > np.array(MAX_BOUND)+EPS_BOUND):
        return False
    return True

def get_ind(low, up):
  use = []
  for i in range(len(low)):
    use.append(numpy.random.uniform(low[i], up[i]))
  return use

def get_ind_valid(low, up, population):
  use = []
  for i in range(len(population)):
    feasible_ind = numpy.array(population[i])
    feasible_ind = numpy.maximum(numpy.array(low), feasible_ind)
    feasible_ind = numpy.minimum(numpy.array(up), feasible_ind)
    use.append(feasible_ind)
  return use

# Problem size
N = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)


low = [-1, -1, -1, -1, -1]
up = [1, 1, 1, 1, 1]
toolbox = base.Toolbox()
toolbox.register("getInd", get_ind, low, up)
toolbox.register("getIndVaild", get_ind_valid, low, up)
toolbox.register("evaluate", benchmarks.rastrigin)
#toolbox.decorate("evaluate", tools.ClosestValidPenalty(valid, closest_feasible, 1.0, distance))
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", numpy.min, axis=0)
stats.register("max", numpy.max, axis=0)
logbook = tools.Logbook()
logbook.header = ["gen", "min", "max"] 


/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [39]:
def main():
    strategy = cma.Strategy(centroid=toolbox.getInd(), sigma=1.0)
    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("update", strategy.update)

    for gen in range(5):
        # Generate a new population
        population = toolbox.generate()
        #print("population: ",population)
        getIndVailds = toolbox.getIndVaild(population)
        #print("getIndVailds: ",getIndVailds)
        populations = [creator.Individual(x) for x in getIndVailds]
        
        # Evaluate the individuals
        fitnesses = toolbox.map(toolbox.evaluate, populations)
        for ind, fit in zip(populations, fitnesses):
            ind.fitness.values = fit
        record = stats.compile(populations)
        logbook.record(gen=gen, nevals=len(population), **record)
        print(logbook.stream)
        best = tools.selBest(population,1)[0]
    print("best",best)
    print("best_val",toolbox.evaluate(best))


In [40]:
if __name__ == "__main__":
    solutions = main()

gen	min          	max          
0  	[22.56473411]	[54.81164008]
1  	[20.39643486]	[50.37000771]
2  	[28.99373027]	[65.56092835]
3  	[8.10267679] 	[64.52933104]
4  	[11.91367079]	[56.23778788]
best [-1.4430322210716933, 0.8818942212099359, 0.827371146293927, -0.3132594451193559, 0.10925498728881444]
best_val (47.114105866229515,)


In [41]:
toolbox.decorate("evaluate", tools.ClosestValidPenalty(valid, closest_feasible,2, distance))
def main():
    strategy = cma.Strategy(centroid=toolbox.getInd(), sigma=1.0)
    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("update", strategy.update)

    for gen in range(5):
        population = toolbox.generate()
        # Evaluate the individuals
        fitnesses = toolbox.map(toolbox.evaluate, population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
        record = stats.compile(population)
        logbook.record(gen=gen, nevals=len(population), **record)
        print(logbook.stream)
    best = tools.selBest(population,1)[0]
    print("best",best)
    print("best_val",toolbox.evaluate(best))

if __name__ == "__main__":
    solutions = main()

0  	[7.75083428] 	[65.57076179]
1  	[20.09510234]	[52.07875044]
2  	[8.12651007] 	[50.06655111]
3  	[25.6366362] 	[57.94808137]
4  	[8.17738627] 	[59.83846911]
best [1.5910803408039447, 1.2157046865032006, -1.9843382293855147, 0.9728260626095173, -1.4218638258846403]
best_val (8.177386267812231,)


In [43]:
toolbox.decorate("evaluate", tools.ClosestValidPenalty(valid, closest_feasible,0.1, distance))
def main():
    strategy = cma.Strategy(centroid=toolbox.getInd(), sigma=1.0)
    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("update", strategy.update)

    for gen in range(10):
        population = toolbox.generate()
        # Evaluate the individuals
        fitnesses = toolbox.map(toolbox.evaluate, population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
        record = stats.compile(population)
        logbook.record(gen=gen, nevals=len(population), **record)
        print(logbook.stream)
    best = tools.selBest(population,1)[0]
    print("best",best)
    print("best_val",toolbox.evaluate(best))

if __name__ == "__main__":
    solutions = main()

0  	[21.14520591]	[72.93945122]
1  	[5.51240676] 	[47.25035463]
2  	[5.60370357] 	[65.65165779]
3  	[27.34085283]	[78.63763131]
4  	[35.84904438]	[62.46345513]
5  	[28.18366613]	[62.23394979]
6  	[9.08635625] 	[76.46080834]
7  	[5.58175702] 	[73.2556438] 
8  	[4.85853843] 	[46.17632109]
9  	[14.09754167]	[67.32993867]
best [-0.05144950813599075, -1.2252172826175787, 1.4336177861944943, 1.2586199876032103, 0.25764204480012004]
best_val (14.097541666322357,)
